In [1]:
import scanpy as sc
import os
import anndata
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from scanpy.metrics.specificity.plot import marker_genes_distribution, one_v_max_genelist
import numpy as np
from scanpy.metrics.specificity.get_data import get_average_celltype_counts, get_spe
from scanpy.metrics.specificity.compute import one_v_max_matrix
from joblib import dump, load
import pickle

Load data

In [179]:
DATA_PATH = r'data/'
MODEL_PATH = r'models/'

In [198]:
count_file = 'lung.cellxgene.h5ad'
adata = anndata.read_h5ad(DATA_PATH + count_file)
adata.obs['Celltypes_updated_July_2020'].cat.categories

Index(['Alveolar_Type1', 'Alveolar_Type2', 'B_cell_mature', 'B_cell_naive',
       'Basal', 'Blood_vessel', 'Ciliated', 'DC_1', 'DC_2',
       'DC_Monocyte_Dividing', 'DC_activated', 'DC_plasmacytoid', 'Fibroblast',
       'Lymph_vessel', 'Macrophage_Dividing', 'Macrophage_MARCOneg',
       'Macrophage_MARCOpos', 'Mast_cells', 'Monocyte', 'Muscle_cells', 'NK',
       'NK_Dividing', 'Plasma_cells', 'Secretory_club', 'T_CD4', 'T_CD8_CytT',
       'T_cells_Dividing', 'T_regulatory'],
      dtype='object')

In [181]:
adata.obs['Celltypes_updated_July_2020'].value_counts()

NK                      10418
T_CD4                    7211
Monocyte                 6067
T_CD8_CytT               5755
Alveolar_Type2           4826
Fibroblast               4624
Macrophage_MARCOpos      4512
Mast_cells               2911
Blood_vessel             2694
DC_2                     1605
Macrophage_MARCOneg       937
B_cell_naive              900
Muscle_cells              839
B_cell_mature             651
T_regulatory              481
Ciliated                  466
Alveolar_Type1            346
Lymph_vessel              337
Plasma_cells              287
DC_Monocyte_Dividing      242
Secretory_club            179
DC_1                      163
NK_Dividing               130
DC_activated              122
Macrophage_Dividing        94
Basal                      89
T_cells_Dividing           88
DC_plasmacytoid            46
Name: Celltypes_updated_July_2020, dtype: int64

In [182]:
def find_max_celltype(gene,adata,partition_key="CellType"):
    get_average_celltype_counts(adata , partition_key = partition_key)
    gene_index = int(np.where(adata.var.index == gene)[0])
    max_celltype_index = np.argmax(adata.varm['ave_celltype_counts_' + partition_key][gene_index,:])
    max_celltype = list(adata.obs[partition_key].cat.categories)[max_celltype_index]
    return(max_celltype)

def full_spe_matrix(adata, partition_key: str = 'CellType') :
    shannon = get_spe(adata, spe_metric='shannon', partition_key=partition_key)
    tau = get_spe(adata, spe_metric='tau', partition_key=partition_key)
    gini = get_spe(adata, spe_metric='gini', partition_key=partition_key)
    one_v_max = one_v_max_matrix(adata,partition_key=partition_key)
    spe_matrix = pd.DataFrame(columns=['gene',
                                       'most expressed celltype',
                                       'one_v_max',
                                       'shannon',
                                       'tau',
                                       'gini'])
    for gene in adata.var.index :
        spe_matrix.loc[gene,:] = [gene,
                                  find_max_celltype(gene,adata,partition_key=partition_key),
                                  max(one_v_max[gene]), 
                                  shannon[gene],
                                  tau[gene],
                                  gini[gene]]
    return spe_matrix

In [183]:
spe_Meyer_full = full_spe_matrix(adata=adata, partition_key='Celltypes_updated_July_2020')

C:\Users\ipmc\.conda\envs\scanpypip\lib\site-packages\anndata\_core\anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
C:\Users\ipmc\.conda\envs\scanpypip\lib\site-packages\scanpy\metrics\specificity\compute.py:222: RuntimeWarning: invalid value encountered in double_scalars
  ((np.sum((2 * index - n - 1) * gini_list)) / (n * np.sum(gini_list)))


In [184]:
spe_Meyer_full = spe_Meyer_full.replace([np.inf, -np.inf], np.nan).dropna()

In [185]:
spe_Meyer = spe_Meyer_full.iloc[:,2:]

In [186]:
spe_Meyer.to_csv(r'data/Meyer_spec.csv')

In [187]:
clf_Deprez_366obs = load(MODEL_PATH + 'clf_train_Deprez_366obs.joblib')

In [188]:
spe_type_Meyer_pred = clf_Deprez_366obs.predict(spe_Meyer)
spec_dict={'0':'equirep','1':'low','2':'multi-spec','3':'high','4':'unique'}
spe_Meyer['spe_type'] = pd.Series(spe_type_Meyer_pred, index = spe_Meyer.index).apply(str).replace(spec_dict)

In [189]:
spe_Meyer

,one_v_max,shannon,tau,gini,spe_type
RP11-34P13.7,3.120907,0.426032,0.956376,0.753801,high
RP11-34P13.8,6.465109,0.124540,0.991510,0.946633,unique
FO538757.3,1.051696,0.168197,0.962499,0.926160,multi-spec
FO538757.2,1.064265,0.681523,0.403931,0.184754,equirep
AP006222.2,1.590516,0.655178,0.718644,0.334584,low
...,...,...,...,...,...
AL354822.1,1.177372,0.600569,0.726390,0.493432,equirep
AC004556.1,1.195818,0.611275,0.732075,0.444289,equirep
AC233755.2,3.009538,0.151347,0.985766,0.940870,high
AC233755.1,31.902023,0.058257,0.997629,0.957543,unique


In [191]:
adata

AnnData object with n_obs × n_vars = 57020 × 25204
    obs: 'Donor', 'Time', 'donor_time', 'organ', 'patient', 'sample', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Celltypes_GenomeBiol_2019', 'Celltypes_updated_July_2020'
    var: 'gene_ids-HCATisStab7509734', 'gene_ids-HCATisStab7509735', 'gene_ids-HCATisStab7509736', 'gene_ids-HCATisStab7587202', 'gene_ids-HCATisStab7587205', 'gene_ids-HCATisStab7587208', 'gene_ids-HCATisStab7587211', 'gene_ids-HCATisStab7646032', 'gene_ids-HCATisStab7646033', 'gene_ids-HCATisStab7646034', 'gene_ids-HCATisStab7646035', 'gene_ids-HCATisStab7659968', 'gene_ids-HCATisStab7659969', 'gene_ids-HCATisStab7659970', 'gene_ids-HCATisStab7659971', 'gene_ids-HCATisStab7747197', 'gene_ids-HCATisStab7747198', 'gene_ids-HCATisStab7747199', 'gene_ids-HCATisStab7747200', 'n_cells', 'shannon_Celltypes_updated_July_2020', 'tau_Celltypes_updated_July_2020', 'gini_Celltypes_updated_July_2020'
    uns: 'Celltypes_GenomeBiol_2019_colors', 'Celltypes_colors', 'Cellty

Choose your Markers

In [150]:
marker_file = 'Marker_cells_Deprez_CT_Meyer.csv'
markers = pd.read_csv(DATA_PATH + marker_file, sep=';')
celltypes = markers.columns

In [151]:
marker_file = 'Markers_Meyer_CT_Deprez.csv'
markers = pd.read_csv(DATA_PATH + marker_file, sep=';')
celltypes = markers.columns

In [152]:
ROOTDIR = r''

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
markers = load_obj(DATA_PATH + r'Markers_Meyer')
markers = pd.DataFrame.from_dict(markers, orient='index').T
celltypes = markers.columns

In [153]:
marker_file = 'Markers_Meyer_df_CT_July2020.csv'
markers = pd.read_csv(DATA_PATH + marker_file, sep=';')
celltypes = markers.columns

Visualization - celltype

In [154]:
dropdown_celltype = widgets.Dropdown(
    options=celltypes,
    value='Basal',
    description='Celltypes_updated_July_2020:',
    disabled=False,
)

In [155]:
output_marker = widgets.Output()

In [156]:
def show_spec_distrib(change):
    if change.new not in adata.obs['Celltypes_updated_July_2020'].cat.categories :
        print(f'{change.new} not found in this dataset identified celltypes')
    else :
        output_marker.clear_output()
        with output_marker :
            spec_markers=markers[change.new].dropna()
            found_markers=spec_markers[spec_markers.isin(adata.var_names)]
            not_found=spec_markers[~spec_markers.isin(adata.var_names)]
            if not not_found.empty:
                print('Gene(s) not found : ' + ','.join(not_found))
            print(spe_Meyer.loc[found_markers,'spe_type'])
            marker_genes_distribution(adata=adata,
                                      gene_list=spec_markers,
                                      celltype=change.new,
                                      partition_key='Celltypes_updated_July_2020')
            one_v_max_genelist(adata=adata,
                               gene_list=spec_markers,
                               partition_key='Celltypes_updated_July_2020')
            sc.pl.umap(adata=adata, color=found_markers, color_map = 'jet')

In [157]:
dropdown_celltype.observe(show_spec_distrib,names='value')
display(dropdown_celltype)

Dropdown(description='Celltypes_updated_July_2020:', index=9, options=('Macrophage_MARCOpos', 'NK', 'Macrophag…

In [158]:
display(output_marker)

Output()

Vizualisation - genes

In [192]:
combobox_genes = widgets.Combobox(
    options=list(adata.var.index),
    placeholder='Type a gene',
    description='Gene:',
    ensure_option=True,
    disabled=False,
)

In [193]:
output_gene=widgets.Output()

In [194]:
def show_spec_gene(change):
    output_gene.clear_output()
    with output_gene :
        max_celltype = find_max_celltype(gene=change.new,adata=adata,partition_key="Celltypes_updated_July_2020")
        print('Specificity type : ' + str(spe_Meyer.loc[change.new,'spe_type']) )
        marker_genes_distribution(adata=adata,
                                  gene_list=[change.new],
                                  celltype=max_celltype,
                                  partition_key='Celltypes_updated_July_2020')
        one_v_max_genelist(adata=adata,
                           gene_list=[change.new],
                           partition_key='Celltypes_updated_July_2020')
        sc.pl.umap(adata=adata, color=change.new, color_map = 'jet')

In [195]:
combobox_genes.observe(show_spec_gene,names='value')
display(combobox_genes)

Combobox(value='', description='Gene:', ensure_option=True, options=('RP11-34P13.7', 'RP11-34P13.8', 'FO538757…

In [196]:
display(output_gene)

Output()

In [ ]:
barbry = pd.melt(markers).dropna()
barbry = barbry.rename(columns={'variable':'CellType','value':'gene'})

In [ ]:
barbry['Proposed by'] = 'Barbry'

In [ ]:
barbry.index = barbry['gene']

In [ ]:
barbry = barbry.join(spe_Meyer)
barbry = barbry.dropna()

In [ ]:
barbry = barbry.dropna()

In [ ]:
barbry = barbry.rename(columns = {'one_v_max' : 'one_v_max_Meyer', 'shannon' : 'shannon_Meyer', 'tau' : 'tau_Meyer', 'gini' : 'gini_Meyer', 'spe_type':'spe_type_Meyer'} )

In [ ]:
barbry.to_csv('Barb_in_Meyer.csv')

In [ ]:
meyer = pd.melt(markers).dropna()
meyer = meyer.rename(columns={'variable':'CellType','value':'gene'})

In [ ]:
meyer['Proposed by'] = 'Meyer'

In [ ]:
meyer.index = meyer['gene']

In [ ]:
meyer = meyer.join(spe_Meyer)
meyer = meyer.dropna()

In [ ]:
meyer = meyer.dropna()

In [ ]:
meyer = meyer.rename(columns = {'one_v_max' : 'one_v_max_Meyer', 'shannon' : 'shannon_Meyer', 'tau' : 'tau_Meyer', 'gini' : 'gini_Meyer', 'spe_type':'spe_type_Meyer'} )

In [ ]:
meyer.to_csv('Meyer_in_Meyer.csv')

In [ ]:
all_Meyer = pd.concat([meyer,barbry])

In [ ]:
all_Meyer.to_csv('All_in_Meyer.csv')

In [ ]:
barbry

In [ ]:
pred_all_Barbry = pd.read_csv(r'data/genes_spec_pred_V1',sep=',',index_col='gene')
pred_all_Barbry.columns += '_Barbry'

pred_all_Meyer = spe_Meyer.copy()
pred_all_Meyer.index.name = 'gene'
pred_all_Meyer.columns += '_Meyer'

In [ ]:
pred_all_Barbry_Meyer = pred_all_Barbry.join(pred_all_Meyer, how = 'inner')

In [126]:
pred_all_Barbry_Meyer.to_csv('spec_all_genes_Barbry_Meyer.csv')

In [ ]:
pred_all_Barbry_Meyer = pd.read_csv('spec_all_genes_Barbry_Meyer.csv', index_col = 'gene')

In [ ]:
pred_all_Barbry_Meyer.columns

In [124]:
pred_all_Barbry_Meyer=pred_all_Barbry_Meyer[['spe_type_Barbry','most expressed celltype_Barbry','spe_type_Meyer','most expressed celltype_Meyer','one_v_max_Barbry', 'shannon_Barbry', 'tau_Barbry', 'gini_Barbry', 'one_v_max_Meyer', 'shannon_Meyer', 'tau_Meyer','gini_Meyer']]

In [ ]:
pred_all_Barbry_Meyer

In [ ]:
meyer_spe=full_spe_matrix(adata,partition_key='Celltypes_updated_July_2020')

In [ ]:
count_file = 'HCA_Barbry_Grch38_Raw_filter_Norm.h5ad'
barbry = anndata.read_h5ad(DATA_PATH + count_file)

In [ ]:
barbry_spe = full_spe_matrix(barbry,partition_key='CellType')

In [115]:
barbry_spe.columns += '_Barbry'

meyer_spe.index.name = 'gene'
meyer_spe.columns += '_Meyer'
pred_all_Barbry_Meyer = meyer_spe.join(barbry_spe, how = 'inner')

In [112]:
meyer_spe = meyer_spe.replace([np.inf, -np.inf], np.nan).dropna()
barbry_spe = barbry_spe.replace([np.inf, -np.inf], np.nan).dropna()

spe_type_meyer_pred = clf_Deprez_366obs.predict(meyer_spe[['one_v_max','shannon','tau','gini']])
spe_type_barbry_pred = clf_Deprez_366obs.predict(barbry_spe[['one_v_max','shannon','tau','gini']])

spec_dict={'0':'equirep','1':'low','2':'multi-spec','3':'high','4':'unique'}
meyer_spe['spe_type'] = pd.Series(spe_type_meyer_pred, index = meyer_spe.index).apply(str).replace(spec_dict)
barbry_spe['spe_type'] = pd.Series(spe_type_barbry_pred, index = barbry_spe.index).apply(str).replace(spec_dict)

In [100]:
pred_all_Barbry_Meyer

,most expressed celltype_Meyer,one_v_max_Meyer,shannon_Meyer,tau_Meyer,gini_Meyer,spe_type_Meyer,one_v_max_Barbry,shannon_Barbry,tau_Barbry,gini_Barbry,spe_type_Barbry
gene,,,,,,,,,,,
LINC00115,Secretory_club,1.339706,0.657481,0.612519,0.323852,equirep,1.226817,0.628045,0.626468,0.399453,equirep
FAM41C,B_cell_naive,1.198358,0.648717,0.634104,0.349967,equirep,1.125282,0.637185,0.663437,0.393725,equirep
NOC2L,Basal,1.174187,0.680950,0.478723,0.190852,equirep,1.463043,0.673486,0.569871,0.219136,equirep
PLEKHN1,T_regulatory,1.884708,0.557493,0.872122,0.596108,low,1.832154,0.519458,0.867355,0.665310,multi-spec
PERM1,Secretory_club,21.341028,0.168331,0.992186,0.921158,unique,1.040512,0.442806,0.854686,0.765738,multi-spec
...,...,...,...,...,...,...,...,...,...,...,...
AC007325.4,Ciliated,3.987447,0.529776,0.933019,0.605659,high,3.163253,0.538201,0.907686,0.615957,high
AC007325.2,Basal,2.505832,0.353413,0.948815,0.848981,high,1.077784,0.414222,0.886558,0.792062,multi-spec
AL354822.1,Ciliated,1.177372,0.600569,0.726390,0.493432,equirep,1.268024,0.604515,0.729433,0.467284,equirep


In [119]:
meyer_spe=meyer_spe.drop('gene_Meyer',axis=1)
barbry_spe=barbry_spe.drop('gene_Barbry',axis=1)

In [88]:
spe_type_meyer_pred = clf_Deprez_366obs.predict(meyer_spe[['one_v_max','shannon','tau','gini']])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [95]:
meyer_spe.replace([np.inf, -np.inf], np.nan).dropna()

,most expressed celltype,one_v_max,shannon,tau,gini
RP11-34P13.7,DC_plasmacytoid,3.120907,0.426032,0.956376,0.753801
RP11-34P13.8,Macrophage_Dividing,6.465109,0.124540,0.991510,0.946633
FO538757.3,Mast_cells,1.051696,0.168197,0.962499,0.926160
FO538757.2,Mast_cells,1.064265,0.681523,0.403931,0.184754
AP006222.2,Muscle_cells,1.590516,0.655178,0.718644,0.334584
...,...,...,...,...,...
AL354822.1,Ciliated,1.177372,0.600569,0.726390,0.493432
AC004556.1,Alveolar_Type1,1.195818,0.611275,0.732075,0.444289
AC233755.2,Plasma_cells,3.009538,0.151347,0.985766,0.940870
AC233755.1,Plasma_cells,31.902023,0.058257,0.997629,0.957543


In [104]:
barbry_spe['spe_type'] = pd.Series(spe_type_barbry_pred, index = barbry_spe.index).apply(str).replace(spec_dict)

In [123]:
pred_all_Barbry_Meyer.drop('gene_Meyer',axis=1)

,most expressed celltype_Meyer,one_v_max_Meyer,shannon_Meyer,tau_Meyer,gini_Meyer,spe_type_Meyer,gene_Barbry,most expressed celltype_Barbry,one_v_max_Barbry,shannon_Barbry,tau_Barbry,gini_Barbry,spe_type_Barbry
LINC00115,Secretory_club,1.339706,0.657481,0.612519,0.323852,equirep,LINC00115,PNEC,1.226817,0.628045,0.626468,0.399453,equirep
FAM41C,B_cell_naive,1.198358,0.648717,0.634104,0.349967,equirep,FAM41C,Deuterosomal,1.125282,0.637185,0.663437,0.393725,equirep
NOC2L,Basal,1.174187,0.680950,0.478723,0.190852,equirep,NOC2L,Precursor,1.463043,0.673486,0.569871,0.219136,equirep
PLEKHN1,T_regulatory,1.884708,0.557493,0.872122,0.596108,low,PLEKHN1,Suprabasal N,1.832154,0.519458,0.867355,0.665310,low
PERM1,Secretory_club,21.341028,0.168331,0.992186,0.921158,unique,PERM1,Suprabasal,1.040512,0.442806,0.854686,0.765738,multi-spec
...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC007325.4,Ciliated,3.987447,0.529776,0.933019,0.605659,high,AC007325.4,Deuterosomal,3.163253,0.538201,0.907686,0.615957,high
AC007325.2,Basal,2.505832,0.353413,0.948815,0.848981,high,AC007325.2,Cycling Basal,1.077784,0.414222,0.886558,0.792062,multi-spec
AL354822.1,Ciliated,1.177372,0.600569,0.726390,0.493432,equirep,AL354822.1,Ionocyte,1.268024,0.604515,0.729433,0.467284,equirep
AC004556.1,Alveolar_Type1,1.195818,0.611275,0.732075,0.444289,equirep,AC004556.1,Precursor,1.420643,0.578837,0.840537,0.550767,multi-spec


In [130]:
new_selec=pred_all_Barbry_Meyer.loc[['FABP4','SPARCL1','CD69','CCL5','JUN','FCER1G','CXCL17','SCGB1A1','CPA3','SCGB3A1','CENPF','ALDH1A3','SCGB3A2','RHOB','CD74','ZFP36L2','IFI27','HPGD','MSLN','CD9','CD52'],:]

In [132]:
new_selec.to_csv('new_genes.csv')